# Chest X-Ray Multi‑Class Project — Role Notebook

**Dataset:** Kaggle “Lungs Disease Dataset (4 types)” by Omkar Manohar Dalvi  
**Classes:** Normal, Bacterial Pneumonia, Viral Pneumonia, COVID‑19, Tuberculosis

> Use this notebook in **Google Colab**. If you’re running locally, adapt the Drive mount steps accordingly.

## Role — Member 3: Model Architect

**Responsibilities**  
- Build a **baseline CNN**  
- Configure **transfer learning** models: MobileNetV2, EfficientNetB0, Xception  
- Replace classification heads for 5 classes  
- Provide model summaries and parameter counts

## Environment & Paths

- The code below mounts Google Drive (for persistence) and prepares base paths.  
- Set `DATASET_DIR` to where the extracted dataset resides (after Kaggle download).

## Model Design Notes

- Use global average pooling + dropout + softmax head  
- Start with frozen backbone, then unfreeze top blocks for fine‑tuning  
- Compile with Adam and `CategoricalCrossentropy`

In [ ]:
# === Colab & Paths ===
import os, sys, glob, json, random, shutil, time
from pathlib import Path

# If in Colab, mount Drive (safe to run elsewhere; it will just fail silently)
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    IN_COLAB = True
except Exception as e:
    print("Not running on Colab or Drive not available:", e)
    IN_COLAB = False

# Project root inside Drive (you can change this)
PROJECT_ROOT = Path('/content/drive/MyDrive/Chest_XRay_Project')
PROJECT_ROOT.mkdir(parents=True, exist_ok=True)

# Where the dataset will live (after download & unzip). Adjust as needed.
DATASET_DIR = PROJECT_ROOT / 'lungs_dataset'
OUTPUTS_DIR = PROJECT_ROOT / 'outputs'
MODELS_DIR = PROJECT_ROOT / 'models'
REPORTS_DIR = PROJECT_ROOT / 'reports'

for p in [OUTPUTS_DIR, MODELS_DIR, REPORTS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATASET_DIR :", DATASET_DIR)
print("OUTPUTS_DIR :", OUTPUTS_DIR)
print("MODELS_DIR  :", MODELS_DIR)
print("REPORTS_DIR :", REPORTS_DIR)

In [ ]:
import tensorflow as tf, json
from tensorflow.keras import layers, models

with open(PROJECT_ROOT / 'classes.json') as f:
    CLASS_NAMES = json.load(f)
NUM_CLASSES = len(CLASS_NAMES)
IMG_SIZE = (224, 224)

# === Utility: make head ===
def make_head(x, num_classes):
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    return out

# === Baseline CNN ===
def build_baseline(input_shape=(224,224,3), num_classes=NUM_CLASSES):
    inp = layers.Input(shape=input_shape)
    x = layers.Conv2D(32,3,activation='relu',padding='same')(inp)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(64,3,activation='relu',padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128,3,activation='relu',padding='same')(x)
    x = layers.MaxPool2D()(x)
    out = make_head(x, num_classes)
    model = models.Model(inp, out, name='Baseline_CNN')
    return model

# === Transfer Learning Factory ===
def build_tl(backbone_name='MobileNetV2', input_shape=(224,224,3), num_classes=NUM_CLASSES, trainable=False):
    if backbone_name == 'MobileNetV2':
        base = tf.keras.applications.MobileNetV2(include_top=False, input_shape=input_shape, weights='imagenet')
        preprocess = tf.keras.applications.mobilenet_v2.preprocess_input
    elif backbone_name == 'EfficientNetB0':
        base = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')
        preprocess = tf.keras.applications.efficientnet.preprocess_input
    elif backbone_name == 'Xception':
        base = tf.keras.applications.Xception(include_top=False, input_shape=input_shape, weights='imagenet')
        preprocess = tf.keras.applications.xception.preprocess_input
    else:
        raise ValueError("Unknown backbone")

    base.trainable = trainable
    inp = layers.Input(shape=input_shape)
    x = preprocess(inp)
    x = base(x, training=False)
    out = make_head(x, num_classes)
    model = models.Model(inp, out, name=f'{backbone_name}_TL')
    return model

# === Build and summarize models ===
models_to_build = [
    ('Baseline', build_baseline),
    ('MobileNetV2', lambda: build_tl('MobileNetV2')),
    ('EfficientNetB0', lambda: build_tl('EfficientNetB0')),
    ('Xception', lambda: build_tl('Xception', input_shape=(299,299,3))),
]

for name, fn in models_to_build:
    try:
        m = fn()
        m.summary()
        # Save initial architecture JSON
        with open(MODELS_DIR / f'{name}_architecture.json', 'w') as f:
            f.write(m.to_json())
    except Exception as e:
        print(f"Failed to build {name}:", e)

In [ ]:
# === Export a chosen model for training (frozen backbone) ===
chosen = build_tl('EfficientNetB0')
chosen.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
               loss='categorical_crossentropy',
               metrics=['accuracy'])
chosen_json = chosen.to_json()
with open(MODELS_DIR / 'model_for_training.json', 'w') as f:
    f.write(chosen_json)
print("Saved model_for_training.json")